In [46]:
"""
SEE: https://github.com/sudoswap/lssvm/blob/main/src/LSSVMPair.sol
Events to query for are:
  1. SwapNFTInPair()
    - emitted by LSSVMPair.swapNFTsForToken()  (sell order)
    - has no data so need to look at Transfer event for volume
  2. SwapNFTOutPair()
    - emitted by LSSVMPair.swapTokenForAnyNFTs() (buy order)
    - has no data so need to look at Transfer event for volume
  3. SpotPriceUpdate(uint128 newSpotPrice)
    - emitted by LSSVMPair on spot price changes: use to track price changes
"""

import pandas as pd
import typing as tp

from ape import accounts, config, chain, Contract, networks, Project, project
from pathlib import PosixPath

In [28]:
networks.parse_network_choice('ethereum:mainnet:alchemy').__enter__()

<alchemy chain_id=1>

In [29]:
current_block = chain.blocks[-1].number
current_block

15462779

In [30]:
sudo_router = Contract("0x2B2e8cDA09bBA9660dCA5cB6233787738Ad68329")
sudo_router

<LSSVMRouter 0x2B2e8cDA09bBA9660dCA5cB6233787738Ad68329>

In [31]:
sudo_factory = Contract("0xb16c1342E617A5B6E4b631EB114483FDB289c0A4")
sudo_factory

<LSSVMPairFactory 0xb16c1342E617A5B6E4b631EB114483FDB289c0A4>

In [32]:
sudo_factory.NewPair

NewPair(address poolAddress)

In [33]:
# use address of top xmons pool to check is pool thru factory
sudo_factory.isPair("0x5caf332dca4e6c9e69d52f320c21e74845353db0", 0)

True

In [10]:
sudo_pair_xmons = Contract("0x5caf332dca4e6c9e69d52f320c21e74845353db0")
sudo_pair_xmons

<LSSVMPair 0x5caF332DcA4e6C9E69d52F320c21e74845353db0>

In [11]:
# utility functions to unfold event arguments into separate columns
def unfold_event_args(key: str, event_args: dict) -> tp.Any:
    return event_args[key]

def unfold(df: pd.DataFrame):
    for key in df.iloc[-1]['event_arguments'].keys():
        df[key] = df['event_arguments'].apply(lambda ev: unfold_event_args(key, ev))

In [12]:
%time query_pairs = sudo_factory.NewPair.query("*", start_block=current_block - 3000)  # last 10 hr
query_pairs

INFO: Cache database has not been initialized
CPU times: user 339 ms, sys: 46.8 ms, total: 386 ms
Wall time: 6.59 s


,event_name,contract_address,event_arguments,transaction_hash,block_number,block_hash,log_index,transaction_index
0,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x68cB4C8aFC32e8f36d30B501c8C...,0xe08d8aaf4a86634924de5aecf9165d1b6d8030fb8e0f...,15459760,0x02b32f06a997a39597f7f4910b6d0f64bfeee2f5b89d...,90,39
1,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x9B66B9DEA89028c37dA710f096B...,0x0a7b6b0655c7f25521850729fe5a8e475706ce39e347...,15459770,0xd3b72f383daf2cdfcb1bb767f178673ca676605cb480...,173,144
2,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x47d1751AED9F7F657d875e1Fc91...,0xe0719c0cf275380699b0bfc6eccadca14b1a64268a84...,15459770,0xd3b72f383daf2cdfcb1bb767f178673ca676605cb480...,177,145
3,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x8A77D905F6cBc12b13823a51614...,0x8de062d37988c7eb90fdf0764a713dd6d5b0858624a7...,15459771,0x274bfaa3c4fae70941c3ec28a8d915da3a624d2cdea5...,100,91
4,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x69e3764F5369b8949697E944a3B...,0xc9b20ad73df1cc604fd5113721c5a3c19c136f761860...,15459809,0x1f0a6529539cd1897cff85258b478d5b756ac704bb15...,165,136
...,...,...,...,...,...,...,...,...
92,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x533434fC6143582bD830538EFD1...,0x2e8c264c13e6c1d94357af475a566a3165d1572bc2d9...,15462653,0x0f85a5c5dc1b3ca0b60d833800efa158b56c77f39965...,44,43
93,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x93f5C0E7ee6cA8aEd59A6abf7FA...,0x29edfa0edbaa03554cc6f66bcedeb8cee29d9f9ce799...,15462663,0xae05a8ca5c1ba424376c750549c3dff370277af41246...,36,24
94,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x30361c2f2e8992065e0458F0Bd8...,0x9fc9d82eb0d6e4e8b70b121fee01d84725192f382610...,15462673,0x6e91da2fb1229b25f13212de53bd555b29bb072420ce...,180,82
95,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0xDb6D62C72939b32cf13Fa870cb0...,0xbf865850fa09e9781e9fd0506321b44aaa1a1fb0353f...,15462677,0x63dd69c5d65f95880e34dfe69bdc9904f29aad836526...,32,25


In [13]:
unfold(query_pairs)

In [14]:
query_pairs

,event_name,contract_address,event_arguments,transaction_hash,block_number,block_hash,log_index,transaction_index,poolAddress
0,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x68cB4C8aFC32e8f36d30B501c8C...,0xe08d8aaf4a86634924de5aecf9165d1b6d8030fb8e0f...,15459760,0x02b32f06a997a39597f7f4910b6d0f64bfeee2f5b89d...,90,39,0x68cB4C8aFC32e8f36d30B501c8C4806d0ac84825
1,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x9B66B9DEA89028c37dA710f096B...,0x0a7b6b0655c7f25521850729fe5a8e475706ce39e347...,15459770,0xd3b72f383daf2cdfcb1bb767f178673ca676605cb480...,173,144,0x9B66B9DEA89028c37dA710f096B25844A3905108
2,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x47d1751AED9F7F657d875e1Fc91...,0xe0719c0cf275380699b0bfc6eccadca14b1a64268a84...,15459770,0xd3b72f383daf2cdfcb1bb767f178673ca676605cb480...,177,145,0x47d1751AED9F7F657d875e1Fc91945eD9e554e66
3,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x8A77D905F6cBc12b13823a51614...,0x8de062d37988c7eb90fdf0764a713dd6d5b0858624a7...,15459771,0x274bfaa3c4fae70941c3ec28a8d915da3a624d2cdea5...,100,91,0x8A77D905F6cBc12b13823a516142a6379d676F7D
4,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x69e3764F5369b8949697E944a3B...,0xc9b20ad73df1cc604fd5113721c5a3c19c136f761860...,15459809,0x1f0a6529539cd1897cff85258b478d5b756ac704bb15...,165,136,0x69e3764F5369b8949697E944a3B598355564dc38
...,...,...,...,...,...,...,...,...,...
92,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x533434fC6143582bD830538EFD1...,0x2e8c264c13e6c1d94357af475a566a3165d1572bc2d9...,15462653,0x0f85a5c5dc1b3ca0b60d833800efa158b56c77f39965...,44,43,0x533434fC6143582bD830538EFD1B7B70AD2b455c
93,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x93f5C0E7ee6cA8aEd59A6abf7FA...,0x29edfa0edbaa03554cc6f66bcedeb8cee29d9f9ce799...,15462663,0xae05a8ca5c1ba424376c750549c3dff370277af41246...,36,24,0x93f5C0E7ee6cA8aEd59A6abf7FAB78BbCf2370cd
94,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x30361c2f2e8992065e0458F0Bd8...,0x9fc9d82eb0d6e4e8b70b121fee01d84725192f382610...,15462673,0x6e91da2fb1229b25f13212de53bd555b29bb072420ce...,180,82,0x30361c2f2e8992065e0458F0Bd81336C198ac069
95,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0xDb6D62C72939b32cf13Fa870cb0...,0xbf865850fa09e9781e9fd0506321b44aaa1a1fb0353f...,15462677,0x63dd69c5d65f95880e34dfe69bdc9904f29aad836526...,32,25,0xDb6D62C72939b32cf13Fa870cb01cf360A0C0c74


In [104]:
sudo_pair_xmons.SpotPriceUpdate

SpotPriceUpdate(uint128 newSpotPrice)

In [105]:
%time query_prices_xmons = sudo_pair_xmons.SpotPriceUpdate.query('*', start_block=current_block - 3000)
query_prices_xmons

CPU times: user 289 ms, sys: 46.5 ms, total: 336 ms
Wall time: 8.14 s


,event_name,contract_address,event_arguments,transaction_hash,block_number,block_hash,log_index,transaction_index
0,SpotPriceUpdate,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{'newSpotPrice': 17330097087378640000},0x2946f9b515e1b1144a2a2a7de75e111bdc1700485e16...,15460128,0xbe6bdd7ff1c6ead20430c3ca591031e3cab7165b492d...,79,52
1,SpotPriceUpdate,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{'newSpotPrice': 14830097087378640000},0xc5dd46e77a6a15f76f76b34bebc89c57a0a7436febb7...,15460372,0x46a34b7810110fe9171b49155170a0a8af02658675e5...,190,127
2,SpotPriceUpdate,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{'newSpotPrice': 17330097087378640000},0x15592fc9f37ff3366ca8cec45612d9b4dc6eec6b76ac...,15462678,0xa641413250e702a7032da1803bd91a6a9095ab11e9ec...,58,37


In [106]:
unfold(query_prices_xmons)

In [107]:
query_prices_xmons

,event_name,contract_address,event_arguments,transaction_hash,block_number,block_hash,log_index,transaction_index,newSpotPrice
0,SpotPriceUpdate,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{'newSpotPrice': 17330097087378640000},0x2946f9b515e1b1144a2a2a7de75e111bdc1700485e16...,15460128,0xbe6bdd7ff1c6ead20430c3ca591031e3cab7165b492d...,79,52,17330097087378640000
1,SpotPriceUpdate,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{'newSpotPrice': 14830097087378640000},0xc5dd46e77a6a15f76f76b34bebc89c57a0a7436febb7...,15460372,0x46a34b7810110fe9171b49155170a0a8af02658675e5...,190,127,14830097087378640000
2,SpotPriceUpdate,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{'newSpotPrice': 17330097087378640000},0x15592fc9f37ff3366ca8cec45612d9b4dc6eec6b76ac...,15462678,0xa641413250e702a7032da1803bd91a6a9095ab11e9ec...,58,37,17330097087378640000
